In [ ]:
import soundfile as sf
import librosa
import json
import numpy as np
import pickle
import pandas as pd
from pathlib import Path
import os
from joblib import Parallel, delayed
import torch
from librosa import display
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pqdm.processes import pqdm
from box import Box

In [ ]:
config = Box.from_json("config.json")

In [ ]:
def get_clip_sr(file_path):
    clip, sr_native = af.read(file_path)
    clip = librosa.to_mono(clip)
    clip = clip.astype('float32')
    sr = 32000
    length = np.ceil(len(clip) / sr_native)
    return clip, sr

In [ ]:
def get_melspec(clip, sr, period, IMAGE_WIDTH, IMAGE_HEIGHT, fmin, htk, power, n_fft, display=False):
    sr = SR
    train_aug = True
    length = len(clip)
    if period > length:
        start = np.random.randint(period - length)
        tmp = np.zeros(period, dtype=clip.dtype)
        tmp[start : start + length] = clip
        clip = tmp
        
    n_fft = 1024
    win_length = n_fft#//2
    hop_length = int((len(clip) - win_length + n_fft) / IMAGE_WIDTH) + 1 
    spect = np.abs(librosa.stft(y=clip, n_fft=n_fft, hop_length=hop_length, win_length=win_length))
    if spect.shape[1] < IMAGE_WIDTH:
        #print('too large hop length, len(clip)=', len(clip))
        hop_length = hop_length - 1
        spect = np.abs(librosa.stft(y=clip, n_fft=n_fft, hop_length=hop_length, win_length=win_length))
    if spect.shape[1] > IMAGE_WIDTH:
        spect = spect[:, :IMAGE_WIDTH]
    n_mels = IMAGE_HEIGHT // 2
    if train_aug:
        power = 1.5 + np.random.rand()
        spect = np.power(spect, power)
    else:
        spect = np.square(spect)
    spect = librosa.feature.melspectrogram(S=spect, sr=sr, n_fft=n_fft, n_mels=n_mels, fmin=300, fmax=16000)
    spect = librosa.power_to_db(spect)
    #print(spect.shape)
    spect = resize(spect, (IMAGE_HEIGHT, IMAGE_WIDTH), preserve_range=True, anti_aliasing=True)
    spect = spect - spect.min()
    smax = spect.max()
    if smax >= 0.001:
        spect = spect / smax
    else:
        spect[...] = 0
    if display:
        plt.imshow(spect)
        plt.show()
    # clip, sr = librosa.load(path, sr=None, mono=False)
    return spect

In [ ]:
def ogg2np(ebird, secondary, filename):
    secondary_ebird = secondary
    input_path = Path('../input/birdclef-2022')
    output_path = Path('./')
    input_path = input_path / 'train_audio' / filename
    filename = filename.split('/')[1].split('.')[0]
    pathname = filename.split('/')[0]
    record, sr = get_clip_sr(input_path)
    period = int(np.ceil(PERIOD * sr))
    length = record.shape[0]
    record = get_melspec(record, SR, period, IMAGE_WIDTH, IMAGE_HEIGHT, fmin, htk, power, n_fft)
    start_time = start_time + int(frame_len * sample_rate)
    end_time = end_time + int(frame_len * sample_rate)
    if period == 0:
        filename = '%s_%d.npy' % (filename, 0)
        np.save(output_path / filename, record)    
    else:
        for i in range(int(np.ceil(length/period))):  
            filename_i = '%s_%d.npy' % (filename, i)
            record_i = record[i*period : (i+3)*period]
            np.save( output_path / filename_i, record_i)
        return length, ebird, secondary_ebird, output_path, filename_i
    
    return length, ebird, secondary_ebird, filename